# CrystFEL data processing

CrystFEL Home: http://www.desy.de/~twhite/crystfel/<br>
CrystFEL Reference: T. A. White, R. A. Kirian, A. V. Martin, A. Aquila, K. Nass, A. Barty and H. N. Chapman. "CrystFEL: a software suite for snapshot serial crystallography". J. Appl. Cryst. 45 (2012), p335–341.<br>
Authors: *Tobiasz Jarosiewicz, Thomas Kluyver, Hans Fangohr* European XFEL GmbH <br>

This notebook follows closely Tom White's excellent tutorial: http://www.desy.de/~twhite/crystfel/tutorial.html


## 1. Set up the modules and environment variables

The module commands are site specific. Just make sure that your environment 
- includes the PATH to crystfel application and scripts
- includes the PATH to mosflm or ipmosflm and possibly other indexing software like XDS
- does not override the default python environment

Alternatively use the setup-script coming with the RPM or DEB packages

In [1]:
# module load tools/crystfel   # works on DESY; might differ elsewhere
source /opt/crystfel/bin/crystfel.setup.sh

In [2]:
module unload anaconda3     # It's part of the HUB environment. but could interfere with a python scriplet of the tutorial

The location of the dataset. Here a local clusterfilesystem is used, but any other like davfs would work as well

In [3]:
export datadir=/beegfs/desy/user/schluenz/Crystfel.Bench

Location were output data are being written. For convenience cd there.

In [4]:
export outdir=/beegfs/desy/user/schluenz/OUT && mkdir -p $outdir 

In [5]:
cd $outdir && pwd

/beegfs/desy/user/schluenz/OUT


In [6]:
# make sure to use crystfel from git master, otherwise won't work with mosflm 7.2.2 (or use v7.2.1)
# to quickly make a local copy of mosflm v7.2.1:
#if [[ ! -e mosflm ]]; then
#    curl -s -L http://www.desy.de/~schluenz/crystfel/mosflm-7.2.1.tgz | gzip -dc | tar -xv -C $PWD
#    ln -s mosflm-linux-64-noX11 mosflm
#    ln -s mosflm-linux-64-noX11 ipmosflm
#fi
#export PATH=$PWD:$PATH
which mosflm || which ipmosflm
which indexamajig

/opt/crystfel/bin/mosflm
/opt/crystfel/bin/indexamajig


__Verify if the CrystFEL software is callable:__

In [7]:
indexamajig

You need to provide the input filename (use -i)


: 1

In [8]:
echo exit | mosflm  || echo exit | ipmosflm

<B><FONT COLOR="#FF0000"><!--SUMMARY_BEGIN-->
<html> <!-- CCP4 HTML LOGFILE -->
<hr>
<!--SUMMARY_END--></FONT></B>


 ************ Mosflm version 7.2.1  for IP, CCD and PAD data 10th September 2015  ***********
 Andrew Leslie and Harry Powell, MRC Laboratory of Molecular Biology,
 Hills Road, Cambridge CB2 0QH, UK
 E-mails: andrew@mrc-lmb.cam.ac.uk, harry@mrc-lmb.cam.ac.uk
 References:
 Mosflm: A.G.W. Leslie and H.R. Powell (2007), Evolving Methods for 
         Macromolecular Crystallography, 245, 41-51 ISBN 978-1-4020-6314-5
 New auto-indexing based on DPS: I. Steller,  R. Bolotovsky and M.G. Rossmann
         (1997) J. Appl. Cryst. 30, 1036-1040
 New iMosflm GUI:  T.G.G. Battye, L. Kontogiannis, O. Johnson, H.R. Powell 
         and A.G.W. Leslie.(2011) Acta Cryst. D67, 271-281
--------------------------------------------------------------------------------
      Mosflm run on Tuesday, October 02 2018 at 13:45 by non chieda il mio nome!
      Compiler command: ifort
      Compiler v

## 2. Download sample data and a detector geometry file (skipped due to large size of data)


## 3. Examine the patterns

![Uncorrected Image](http://www.desy.de/~schluenz/crystfel/LCLS_2013_Mar23_r0131_003823_b32b.h5.2.png)Executing hdfsee as below would show such an image. hdfsee is however gtk based requiring x-forwarding and currently can't be embedded into a notebook. That's work in progress.

In [9]:
# hdfsee $datadir/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003823_b32b.h5

![Corrected Image](http://www.desy.de/~schluenz/crystfel/LCLS_2013_Mar23_r0131_003823_b32b.h5.geom.2.png)
_Same thing with a geometry file:_

In [10]:
# hdfsee $datadir/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003823_b32b.h5 -g $datadir/5HT2B-Liu-2013.geom

## 4. Create a list of files to process

In [11]:
find $datadir/cxidb-21-run013* -name 'LCLS*.h5' > files.lst

For many purposes a reduced set of images is fully sufficient (and much faster to process)

In [12]:
head -200 files.lst > files.lst.200
head -500 files.lst > files.lst.500

In [13]:
wc files.lst

  5775   5775 590914 files.lst


In [14]:
head -10 files.lst

/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003844_d09b.h5
/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003851_da07.h5
/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003712_4e09.h5
/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003730_684f.h5
/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003831_bd8d.h5
/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003835_c2c7.h5
/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003900_e66d.h5
/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003823_b32b.h5
/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003832_bf5b.h5
/beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r

## 5. Peak Detection

Running indexamajig without crystal parameters will just produce a list of peak, but won't do indexing or integration. Dont't be surprised if none of the crystals is indicated as indexable.

In [15]:
# we use here 32 threads in parallel. Adjust according to resources available.
# processing 200 images takes still a couple of minutes ... 
indexamajig -i files.lst.200 -g $datadir/5HT2B-Liu-2013.geom --peaks=hdf5 -o tutorial.stream -j 32

No indexing methods specified.  I will try to automatically detect the available methods.
To disable auto-detection of indexing methods, specify which methods to use with --indexing=<methods>.
Use --indexing=none to disable indexing and integration.
No reference unit cell provided.
List of indexing methods:
   0: mosflm-nolatt-nocell      (mosflm - no prior information)
   1: asdf-nolatt-nocell        (asdf - no prior information)
Indexing parameters:
                  Check unit cell parameters: off
                        Check peak alignment: on
                   Refine indexing solutions: on
 Multi-lattice indexing ("delete and retry"): off
                              Retry indexing: on
  12 indexable out of   14 processed (85.7%),   12 crystals so far.   14 images processed since the last message.
  30 indexable out of   40 processed (75.0%),   30 crystals so far.   26 images processed since the last message.
  48 indexable out of   61 processed (78.7%),   48 crystals so far.  

![Corrected Image](http://www.desy.de/~schluenz/crystfel/LCLS_2013_Mar23_r0131_003730_684f.h5.peaks.2.png)
check-peak-detection again invokes hdfsee requiring X-forwarding. That's still work in progress. 
To avoid going through hundreds of images, the first command below selects the peaks for the first 3 images

In [16]:
# just fetch the first 3 images to display
#n=0; while read line; do if [[ $n -lt 3 ]]; then echo "$line"; fi; if [[ "$line" =~ "End chunk" ]]; then n=$(($n+1)); fi; done < tutorial.stream > tutorial.3
#check-peak-detection tutorial.3 -g $datadir/5HT2B-Liu-2013.geom --int-boost=5


## 5. Optimise the peak detection parameters

Please refer to the original tutorial. This step is simply skipped here, as it's instructive but not so much in the context of this notebook.

## 6. Determine the unit cell parameters

__Use mosflm for indexing. No lattice or cell dimensions specified__

In [17]:
indexamajig -i files.lst.200 -g $datadir/5HT2B-Liu-2013.geom --peaks=hdf5 \
            --indexing=mosflm --int-radius=3,4,5 -o tutorial.stream -j 32

No reference unit cell provided.
List of indexing methods:
   0: mosflm-nolatt-nocell      (mosflm - no prior information)
Indexing parameters:
                  Check unit cell parameters: off
                        Check peak alignment: on
                   Refine indexing solutions: on
 Multi-lattice indexing ("delete and retry"): off
                              Retry indexing: on
  14 indexable out of   16 processed (87.5%),   14 crystals so far.   16 images processed since the last message.
  32 indexable out of   43 processed (74.4%),   32 crystals so far.   27 images processed since the last message.
  53 indexable out of   70 processed (75.7%),   53 crystals so far.   27 images processed since the last message.
  67 indexable out of   92 processed (72.8%),   67 crystals so far.   22 images processed since the last message.
  76 indexable out of  111 processed (68.5%),   76 crystals so far.   19 images processed since the last message.
  87 indexable out of  128 processed (6

_See the results:_

In [ ]:
egrep 'Cell|lattice_type' tutorial.stream  | paste - - > cells.lattice.txt
head -10 cells.lattice.txt
# Note listed in nanometers not in Angstroem!

In [ ]:
# cell_explorer tutorial.stream

![Corrected Image](http://www.desy.de/~schluenz/crystfel/cell_explorer_tutorial3_thumb.png)
cell_explorer is another tool based on gtk requiring X-forwarding. 
A primitive scriplet using gnuplot as an alternative gives at least an idea ....

In [18]:
# a bit inconvenient but jupyter uses relative paths from the location of the current notebook
# in my case the notebook was located in $HOME. The images are than created in $HOME/images
# change the command accordingly
cellgnu.sh tutorial.stream $HOME

Lattice:      monoclinic  #indexed:     20   Cell:     62.10   62.44  166.32   89.99   90.79   89.84  
Lattice:    orthorhombic  #indexed:     50   Cell:     61.45  122.71  167.84   89.82   90.10   90.10  
Lattice:       triclinic  #indexed:     64   Cell:     62.95   67.75  168.82   91.14   90.35   64.27  
Generated gnuplot image: /home/schluenz/images/cell.1.png
Generated gnuplot image: /home/schluenz/images/cell.2.png
Generated gnuplot image: /home/schluenz/images/cell.3.png
Generated gnuplot image: /home/schluenz/images/cell.4.png
Generated gnuplot image: /home/schluenz/images/cell.5.png
Generated gnuplot image: /home/schluenz/images/cell.6.png


A | B | C
- | - | - 
![Crystal Cells](http://www.desy.de/~schluenz/crystfel/cell.1.png) | ![Crystal Cells](http://www.desy.de/~schluenz/crystfel/cell.2.png) | ![Crystal Cells](http://www.desy.de/~schluenz/crystfel/cell.3.png)
Alpha | Beta | Gamma
![Crystal Cells](http://www.desy.de/~schluenz/crystfel/cell.4.png) | ![Crystal Cells](http://www.desy.de/~schluenz/crystfel/cell.5.png) | ![Crystal Cells](http://www.desy.de/~schluenz/crystfel/cell.6.png)


## 7. Index the patterns first time


__Create a cell unit file: 5ht2b.lattice__
Have a look at Tom White's original tutorial (http://www.desy.de/~twhite/crystfel/tutorial.html#cellparams) to find out why an orthorombic lattice is a good choice. 

In [19]:
cat <<EOF > $outdir/5ht2b.lattice
CrystFEL unit cell file version 1.0

lattice_type = orthorhombic
centering = C
EOF

In [20]:
# Note: using 32 threads. Please adjust! 
indexamajig -i files.lst.200 -p $outdir/5ht2b.lattice -g $datadir/5HT2B-Liu-2013.geom \
            --peaks=hdf5 --int-radius=3,4,5 -o tutorial.stream --indexing=mosflm -j 32

This is what I understood your unit cell to be:
orthorhombic C, unique axis ?.
Unit cell parameters are not specified.
List of indexing methods:
   0: mosflm-latt-nocell        (mosflm using Bravais lattice type as prior information)
Indexing parameters:
                  Check unit cell parameters: off
                        Check peak alignment: on
                   Refine indexing solutions: on
 Multi-lattice indexing ("delete and retry"): off
                              Retry indexing: on
  11 indexable out of   14 processed (78.6%),   11 crystals so far.   14 images processed since the last message.
Unit cell is too large - will only integrate reflections up to 511th order.
orthorhombic C, unique axis *, right handed.
a      b      c            alpha   beta  gamma
374.94 1323.51 274.07 A     89.21  90.36  90.74 deg
a =  4.107e-09  3.565e-08  1.087e-08 m
b = -9.024e-08 -2.028e-08  9.467e-08 m
c =  1.958e-08 -7.781e-09  1.753e-08 m
a* =  2.803e+06  2.526e+07  8.085e+06 m^-1 (mod

In [21]:
#cell_explorer tutorial.stream

In [23]:
cellgnu.sh tutorial.stream $HOME

Lattice:    orthorhombic  #indexed:    116   Cell:     61.52  122.75  167.45   89.83   90.17   90.08  
Generated gnuplot image: /home/schluenz/images/cell.1.png
Generated gnuplot image: /home/schluenz/images/cell.2.png
Generated gnuplot image: /home/schluenz/images/cell.3.png
Generated gnuplot image: /home/schluenz/images/cell.4.png
Generated gnuplot image: /home/schluenz/images/cell.5.png
Generated gnuplot image: /home/schluenz/images/cell.6.png


## 8. Check and optimise the detector geometry

__Use the 'detector_shift' script (requires python 2.7)__
It contains some python qt-visualization which currently doesn't work in the notebook. Work in progress. But it writes the modified geometry-file nevertheless

In [25]:
detector-shift tutorial.stream $datadir/5HT2B-Liu-2013.geom

Mean shifts: dx = -0.1 mm,  dy = -0.18 mm
Applying corrections to /beegfs/desy/user/schluenz/Crystfel.Bench/5HT2B-Liu-2013.geom, output filename /beegfs/desy/user/schluenz/Crystfel.Bench/5HT2B-Liu-2013-predrefine.geom
Using default resolution (9097.525473 px/m) for panel q0a0
Using default resolution (9097.525473 px/m) for panel q0a0
Using default resolution (9097.525473 px/m) for panel q0a1
Using default resolution (9097.525473 px/m) for panel q0a1
Using default resolution (9097.525473 px/m) for panel q0a2
Using default resolution (9097.525473 px/m) for panel q0a2
Using default resolution (9097.525473 px/m) for panel q0a3
Using default resolution (9097.525473 px/m) for panel q0a3
Using default resolution (9097.525473 px/m) for panel q0a4
Using default resolution (9097.525473 px/m) for panel q0a4
Using default resolution (9097.525473 px/m) for panel q0a5
Using default resolution (9097.525473 px/m) for panel q0a5
Using default resolution (9097.525473 px/m) for panel q0a6
Using default r

  File "/usr/lib64/python2.7/site-packages/matplotlib/backends/backend_qt4agg.py", line 53, in new_figure_manager_given_figure
    canvas = FigureCanvasQTAgg(figure)
  File "/usr/lib64/python2.7/site-packages/matplotlib/backends/backend_qt4agg.py", line 76, in __init__
    FigureCanvasQT.__init__(self, figure)
  File "/usr/lib64/python2.7/site-packages/matplotlib/backends/backend_qt4.py", line 63, in __init__
    _create_qApp()
  File "/usr/lib64/python2.7/site-packages/matplotlib/backends/backend_qt5.py", line 144, in _create_qApp
    raise RuntimeError('Invalid DISPLAY variable')
RuntimeError: Invalid DISPLAY variable


: 1

![Corrected Image](http://www.desy.de/~schluenz/crystfel/detectorshift1.png)

__The updated geometry file is written as "5HT2B-Liu-2013-predrefine.geom".__

In [26]:
indexamajig -i files.lst.200 -g $datadir/5HT2B-Liu-2013-predrefine.geom --peaks=hdf5 \
            --int-radius=3,4,5 -o tutorial.stream --indexing=mosflm- -p 5ht2b.lattice -j 16

This is what I understood your unit cell to be:
orthorhombic C, unique axis ?.
Unit cell parameters are not specified.
List of indexing methods:
   0: mosflm-latt-nocell        (mosflm using Bravais lattice type as prior information)
Indexing parameters:
                  Check unit cell parameters: off
                        Check peak alignment: on
                   Refine indexing solutions: on
 Multi-lattice indexing ("delete and retry"): off
                              Retry indexing: on
   2 indexable out of    3 processed (66.7%),    2 crystals so far.    3 images processed since the last message.
  11 indexable out of   20 processed (55.0%),   11 crystals so far.   17 images processed since the last message.
  22 indexable out of   40 processed (55.0%),   22 crystals so far.   20 images processed since the last message.
  38 indexable out of   62 processed (61.3%),   38 crystals so far.   22 images processed since the last message.
  46 indexable out of   73 processed (63.0

In [28]:
cellgnu.sh tutorial.stream $HOME

Lattice:    orthorhombic  #indexed:    119   Cell:     61.73  122.66  167.51   89.80   90.14   90.09  
Generated gnuplot image: /home/schluenz/images/cell.1.png
Generated gnuplot image: /home/schluenz/images/cell.2.png
Generated gnuplot image: /home/schluenz/images/cell.3.png
Generated gnuplot image: /home/schluenz/images/cell.4.png
Generated gnuplot image: /home/schluenz/images/cell.5.png
Generated gnuplot image: /home/schluenz/images/cell.6.png


## 9. Index the patterns using unit cell parameters

__Update the cell file with results from fitting gaussian functions in cell explorer.__

In [29]:
cat <<EOF > 5ht2b.cell
CrystFEL unit cell file version 1.0

lattice_type = orthorhombic
centering = C
a = 61.73 A
b = 122.66 A
c = 167.51 A
al = 90.0 deg
be = 90.0 deg
ga = 90.0 deg
EOF

__Do the indexing for the full set of images:__

In [30]:
indexamajig -i files.lst -g $datadir/5HT2B-Liu-2013-predrefine.geom --peaks=hdf5 \
            --int-radius=3,4,5 -o tutorial.stream --indexing=mosflm- -p 5ht2b.cell -j 16

This is what I understood your unit cell to be:
orthorhombic C, unique axis ?, right handed.
a      b      c            alpha   beta  gamma
 61.73 122.66 167.51 A     90.00  90.00  90.00 deg
a =  6.173e-09  0.000e+00  0.000e+00 m
b =  7.511e-25  1.227e-08  0.000e+00 m
c =  1.026e-24  1.026e-24  1.675e-08 m
a* =  1.620e+08 -9.919e-09 -9.919e-09 m^-1 (modulus  1.620e+08 m^-1)
b* =  0.000e+00  8.153e+07 -4.992e-09 m^-1 (modulus  8.153e+07 m^-1)
c* =  0.000e+00  0.000e+00  5.970e+07 m^-1 (modulus  5.970e+07 m^-1)
alpha* =  90.00 deg, beta* =  90.00 deg, gamma* =  90.00 deg
Cell representation is crystallographic, direct space.
List of indexing methods:
   0: mosflm-latt-cell          (mosflm using cell parameters and Bravais lattice type as prior information)
Indexing parameters:
                  Check unit cell parameters: on (axis combinations)
                        Check peak alignment: on
                   Refine indexing solutions: on
 Multi-lattice indexing ("delete and retry"): 

1167 indexable out of 1779 processed (65.6%), 1167 crystals so far.   30 images processed since the last message.
1188 indexable out of 1809 processed (65.7%), 1188 crystals so far.   30 images processed since the last message.
1208 indexable out of 1837 processed (65.8%), 1208 crystals so far.   28 images processed since the last message.
1233 indexable out of 1873 processed (65.8%), 1233 crystals so far.   36 images processed since the last message.
1261 indexable out of 1906 processed (66.2%), 1261 crystals so far.   33 images processed since the last message.
1278 indexable out of 1934 processed (66.1%), 1278 crystals so far.   28 images processed since the last message.
1296 indexable out of 1963 processed (66.0%), 1296 crystals so far.   29 images processed since the last message.
1322 indexable out of 1997 processed (66.2%), 1322 crystals so far.   34 images processed since the last message.
1336 indexable out of 2024 processed (66.0%), 1336 crystals so far.   27 images processe

2491 indexable out of 3812 processed (65.3%), 2491 crystals so far.   28 images processed since the last message.
2503 indexable out of 3842 processed (65.1%), 2503 crystals so far.   30 images processed since the last message.
2519 indexable out of 3869 processed (65.1%), 2519 crystals so far.   27 images processed since the last message.
2533 indexable out of 3897 processed (65.0%), 2533 crystals so far.   28 images processed since the last message.
2550 indexable out of 3927 processed (64.9%), 2550 crystals so far.   30 images processed since the last message.
2569 indexable out of 3958 processed (64.9%), 2569 crystals so far.   31 images processed since the last message.
2586 indexable out of 3988 processed (64.8%), 2586 crystals so far.   30 images processed since the last message.
2600 indexable out of 4019 processed (64.7%), 2600 crystals so far.   31 images processed since the last message.
2617 indexable out of 4047 processed (64.7%), 2617 crystals so far.   28 images processe

3432 indexable out of 5678 processed (60.4%), 3432 crystals so far.   26 images processed since the last message.
3445 indexable out of 5705 processed (60.4%), 3445 crystals so far.   27 images processed since the last message.
3455 indexable out of 5731 processed (60.3%), 3455 crystals so far.   26 images processed since the last message.
3470 indexable out of 5757 processed (60.3%), 3470 crystals so far.   26 images processed since the last message.
Waiting for the last patterns to be processed...
Final: 5775 images processed, 3473 had crystals (60.1%), 3473 crystals overall.


__Verify the updated geometry/beam shift correction (?)__

In [31]:
detector-shift tutorial.stream $datadir/5HT2B-Liu-2013-predrefine.geom

Mean shifts: dx = -0.031 mm,  dy = -0.064 mm
Applying corrections to /beegfs/desy/user/schluenz/Crystfel.Bench/5HT2B-Liu-2013-predrefine.geom, output filename /beegfs/desy/user/schluenz/Crystfel.Bench/5HT2B-Liu-2013-predrefine-predrefine.geom
Using default resolution (9097.525473 px/m) for panel q0a0
Using default resolution (9097.525473 px/m) for panel q0a0
Using default resolution (9097.525473 px/m) for panel q0a1
Using default resolution (9097.525473 px/m) for panel q0a1
Using default resolution (9097.525473 px/m) for panel q0a2
Using default resolution (9097.525473 px/m) for panel q0a2
Using default resolution (9097.525473 px/m) for panel q0a3
Using default resolution (9097.525473 px/m) for panel q0a3
Using default resolution (9097.525473 px/m) for panel q0a4
Using default resolution (9097.525473 px/m) for panel q0a4
Using default resolution (9097.525473 px/m) for panel q0a5
Using default resolution (9097.525473 px/m) for panel q0a5
Using default resolution (9097.525473 px/m) for p

    return new_figure_manager_given_figure(num, thisFig)
  File "/usr/lib64/python2.7/site-packages/matplotlib/backends/backend_qt4agg.py", line 53, in new_figure_manager_given_figure
    canvas = FigureCanvasQTAgg(figure)
  File "/usr/lib64/python2.7/site-packages/matplotlib/backends/backend_qt4agg.py", line 76, in __init__
    FigureCanvasQT.__init__(self, figure)
  File "/usr/lib64/python2.7/site-packages/matplotlib/backends/backend_qt4.py", line 63, in __init__
    _create_qApp()
  File "/usr/lib64/python2.7/site-packages/matplotlib/backends/backend_qt5.py", line 144, in _create_qApp
    raise RuntimeError('Invalid DISPLAY variable')
RuntimeError: Invalid DISPLAY variable


: 1

![Corrected Image](http://www.desy.de/~schluenz/crystfel/detector-shift.png)

## 10. Evaluate the quality of the indexing

In [32]:
# requires X11. work in progress. Sample image see below.
#check-near-bragg tutorial.stream -g $datadir/5HT2B-Liu-2013-predrefine.geom

Extra arguments for hdfsee: -g /beegfs/desy/user/schluenz/Crystfel.Bench/5HT2B-Liu-2013-predrefine.geom
Viewing /beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0132/data1/LCLS_2013_Mar23_r0132_010317_e901.h5

(hdfsee:19301): Gtk-WARNING **: cannot open display: 


![Corrected Image](http://www.desy.de/~schluenz/crystfel/check-near-bragg.png)

In [33]:
#check-peak-detection --not-indexed tutorial.stream -g $datadir/5HT2B-Liu-2013-predrefine.geom

Extra arguments for hdfsee: -g /beegfs/desy/user/schluenz/Crystfel.Bench/5HT2B-Liu-2013-predrefine.geom
Not showing /beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0132/data1/LCLS_2013_Mar23_r0132_010317_e901.h5
Not showing /beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0132/data1/LCLS_2013_Mar23_r0132_010306_d8bd.h5
Viewing /beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0131/data1/LCLS_2013_Mar23_r0131_003832_bead.h5

(hdfsee:19313): Gtk-WARNING **: cannot open display: 


In [34]:
c#heck-peak-detection --indexed tutorial.stream -g $datadir/5HT2B-Liu-2013-predrefine.geom

Extra arguments for hdfsee: -g /beegfs/desy/user/schluenz/Crystfel.Bench/5HT2B-Liu-2013-predrefine.geom
Viewing /beegfs/desy/user/schluenz/Crystfel.Bench/cxidb-21-run0132/data1/LCLS_2013_Mar23_r0132_010317_e901.h5

(hdfsee:19320): Gtk-WARNING **: cannot open display: 


## 11. Check for detector saturation

__Run the peakogram-stream script (python 2.7)__

In [35]:
# peakogram-stream -i tutorial.stream

![Corrected Image](http://www.desy.de/~schluenz/crystfel/peakogram.png)

## 12. Merge the intensities


In [36]:
process_hkl -i tutorial.stream -o tutorial.hkl -y 222 --max-adu=7000

5775 images processed, 3473 crystals, 3473 crystals used.


In [38]:
process_hkl -i tutorial.stream -o tutorial.hkl1 -y 222 --max-adu=7000 --even-only

5775 images processed, 3473 crystals, 1736 crystals used.


In [39]:
process_hkl -i tutorial.stream -o tutorial.hkl2 -y 222 --max-adu=7000 --odd-only

5775 images processed, 3473 crystals, 1737 crystals used.


## 13. Merge the data using scaling, partiality and post-refinement


In [40]:
partialator -i tutorial.stream -o tutorial.hkl -y 222 --iterations=1 --model=unity --max-adu=7000

Setting --no-pr because we are not modelling partialities (--model=unity).
5775 images loaded, 3473 crystals.
Initial partiality calculation...
Initial scaling...
Scaling: |==================================================|
Log residual went from 1.747926e+05 to 1.209407e+05, 3473 crystals
Mean B = -4.166610e-20
Scaling: |==================================================|
Log residual went from 1.307111e+05 to 1.162877e+05, 3473 crystals
Mean B = -1.362409e-19
Scaling: |==================================================|
Log residual went from 1.294387e+05 to 1.162979e+05, 3473 crystals
Mean B = -2.344435e-19
Overall CChalf = 0.687687 % (80404 reflections)
Calculating deltaCChalf: |==================================================|
deltaCChalf = 0.294458 ± 5.062702 %
28 bad crystals:
  3445 OK
  21 negative delta CC½
  7 B too big
Residuals:  linear          linear free     log             log free
            2.271087e+03    2.661801e+03    4.228061e+05    2.126564e+04    (3444 cry

In [41]:
head -10 tutorial.hkl

CrystFEL reflection list version 2.0
Symmetry: 222
   h    k    l          I    phase   sigma(I)   nmeas
   0    0    5   -1461.23        -    1368.12      16
   0    0    6   26101.69        -   10218.72      18
   0    0    7    -576.56        -    1185.34      24
   0    0    8   31465.06        -    8693.53      23
   0    0    9    -155.13        -     232.30      25
   0    0   10    2277.30        -    1026.83      30
   0    0   11     120.06        -     391.16      30


In [42]:
# upload and publish on zenodo: in preparation